In [1]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.5/725.5 kB 20.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install Datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 92.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: dill
    Found existing installation: dill 0.4.0
    Uninstalling dill-0.4.0:
      Successfully uninstalled dill-0.4.0
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.18
    Uninstalling multiprocess-0.70.18:
      Successfully uninstalled multiprocess-0.70.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pathos 0.3.4 requires dill>=0.4.0, but you have dill 0.3.8 which is incompatible.
pathos 0.3.4 requires multiprocess>=0.70.18, but you have multiprocess 0.70.16 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [34]:
import llm_ops
import re
import json
from importlib import reload, import_module
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.metrics import roc_auc_score
from datasets import load_dataset

In [7]:

df = load_dataset('gretelai/symptom_to_diagnosis')
training_set = pd.DataFrame(df['train'])


In [ ]:
def accuracy_score(x, y):
    accuracy = 0
    for r, d in zip(x, y):
        j = json.loads(re.sub(r'\}\}', '}', re.sub(r'\{\{', '{', r)))
        if j[self.target] == d:
            accuracy += 1
    return accuracy

In [228]:
promptizer3.output_filter('likely')

0.7

In [210]:
class impromptune:
    def __init__(self, library, train_set, target, output_filter):
#        self.library = __import__(library)
        self.library = import_module(library)
        self.train = train_set
        self.target = target
#        self.task = task
        self.scores = []
        self.custom_ids = []
        self.output_filter = output_filter

    def populate_search_space(self, model, N):
        self.task_prompts = llm_ops.batch_same_prompt(self.library.writer_system, self.library.writer_user, N, .9)
        self.task_prompts['custom_id'] = self.task_prompts['custom_id'].astype(int)
        # probably a faster way to do this
        self.task_prompts['embedding'] = [llm_ops.client.embeddings.create(model='text-embedding-3-small', input = x).data[0].embedding  for x in task_prompts['response']]

    def score_random(self, N):

        self.random_prompts = self.task_prompts.sample(N)

        for c, prompt in zip(self.random_prompts['custom_id'], self.random_prompts['response']):
            user_prompts = [prompt + "\n\n### {} ####\n".format(self.library.task_delimiter) + t for t in self.train['input_text']]
            self.score_df = llm_ops.batch_prompts(self.library.task_system, user_prompts, .01)
            self.score_df ['custom_id'] = self.score_df ['custom_id'].astype(int)

            accuracy = self.output_filter(self.score_df.sort_values('custom_id')['response'], self.train[self.target])

            self.scores.append(accuracy/self.score_df.shape[0])
            self.custom_ids.append(c)

    
            
    def BO(self): 
        
        gpr = GaussianProcessRegressor(kernel = Matern() + WhiteKernel())
        scores_ecdf = ecdf(self.scores)
        
        transformed_scores = np.log(lognorm.ppf(scores_ecdf.cdf.evaluate(self.scores) * .999 + .0005, 1))

        self.scored_prompt_embeddings = []
        scored_custom_ids = []
        unscored_prompt_embeddings = []
        unscored_custom_ids = []
        for x in range(promptizer3.task_prompts.shape[0]):
            if promptizer3.task_prompts['custom_id'].iloc[x] not in promptizer3.custom_ids:
                unscored_prompt_embeddings.append(promptizer3.task_prompts['embedding'].iloc[x])
                unscored_custom_ids.append(promptizer3.task_prompts['custom_id'].iloc[x])
            else:
                self.scored_prompt_embeddings.append(promptizer3.task_prompts['embedding'].iloc[x])
                scored_custom_ids.append(promptizer3.task_prompts['custom_id'].iloc[x])

        
        gpr.fit(self.scored_prompt_embeddings, transformed_scores)

        y_best = max(transformed_scores)
        gpr_predictions, gpr_sigma = gpr.predict(unscored_prompt_embeddings, return_std=True)
        
        best_ei = -10
        best_idx = -1
        
        for m, s in zip(enumerate(gpr_predictions), gpr_sigma):
            ei = np.exp(m[1]+s**2/2)*(1-norm.cdf(y_best, m[0]+s**2, s))
            if ei > best_ei:
                best_ei = ei
                best_idx = m[0]
                
        return best_ei, unscored_custom_ids[best_idx]


    def iterate_score(self, idx):
        # one at at time
        next_prompt = self.task_prompts[self.task_prompts['custom_id'] == idx]['response'].iloc[0]
        user_prompts = [next_prompt + "\n\n### {} ####\n".format(self.library.task_delimiter) + t  for t in self.train['input_text']]
        self.responses = llm_ops.batch_prompts(self.library.task_system, user_prompts, .01)

        self.score_df = llm_ops.batch_prompts(self.library.task_system, user_prompts, .01)
        self.score_df ['custom_id'] = self.score_df ['custom_id'].astype(int)
        accuracy = self.output_filter(self.score_df.sort_values('custom_id')['response'], self.train[self.target])

        self.scores.append(accuracy/self.score_df.shape[0])
        self.custom_ids.append(idx)


In [211]:
promptizer3 = impromptune('prompt_library.medical_diagnosis', pd.DataFrame(df['train']).sample(10), 'disease', llm_ops.probability)
promptizer3.task_prompts = promptizer.task_prompts
promptizer3.custom_ids = [2, 9, 6]
promptizer3.scores = promptizer2.scores
promptizer3.train = promptizer2.train
promptizer3.target = promptizer2.target
promptizer3.prompts = promptizer2.prompts
promptizer3.score_df = promptizer2.score_df

# roc_auc_score(y_test, p)
promptizer3.task_prompts['custom_id'] = promptizer3.task_prompts['custom_id'].astype(int)


In [212]:
promptizer3.task_prompts['embedding'] = [llm_ops.client.embeddings.create(model='text-embedding-3-small', input = x).data[0].embedding  for x in promptizer3.task_prompts['response']]

In [213]:
#promptizer.populate_search_space('gpt-4o', 10)

In [ ]:
#promptizer2.score_random(3)

In [215]:
best_ei, best_next_guess = promptizer3.BO()
print(best_next_guess)

7


In [222]:
#promptizer3.task_prompts['custom_id']
promptizer3.custom_ids


[2, 9, 6, 3]

In [214]:
promptizer3.iterate_score(best_next_guess)

{
  "id": "file-78d6923f17064d0e831bbae4f6da13fe",
  "bytes": 20954,
  "created_at": 1749146368,
  "filename": "batch.jsonl",
  "object": "file",
  "purpose": "batch",
  "status": "processed",
  "expires_at": null,
  "status_details": null
}
Create batch job:

{
  "id": "batch_27d27d8e-f43c-4d2b-b905-2ca2d1a9d63c",
  "completion_window": "24h",
  "created_at": 1749146371,
  "endpoint": "/chat/completions",
  "input_file_id": "file-78d6923f17064d0e831bbae4f6da13fe",
  "object": "batch",
  "status": "validating",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": null,
  "error_file_id": "",
  "errors": null,
  "expired_at": null,
  "expires_at": 1749232768,
  "failed_at": null,
  "finalizing_at": null,
  "in_progress_at": null,
  "metadata": null,
  "output_file_id": "",
  "request_counts": {
    "completed": 0,
    "failed": 0,
    "total": 0
  }
}
Track batch job:

2025-06-05 17:59:31.624190 Batch Id: batch_27d27d8e-f43c-4d2b-b905-2ca2d1a9d63c,  Status: validating
2025

ai_detector
medical_diagnosis
is_rumor
robert_frost

{
  "id": "file-1d8517de65a1421cb9cc573278133fbd",
  "bytes": 17860,
  "created_at": 1749134661,
  "filename": "batch_same_prompt.jsonl",
  "object": "file",
  "purpose": "batch",
  "status": "processed",
  "expires_at": null,
  "status_details": null
}
Create batch job:

{
  "id": "batch_455d9b12-478c-4e8c-ae56-e3d1f48eca66",
  "completion_window": "24h",
  "created_at": 1749134664,
  "endpoint": "/chat/completions",
  "input_file_id": "file-1d8517de65a1421cb9cc573278133fbd",
  "object": "batch",
  "status": "validating",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": null,
  "error_file_id": "",
  "errors": null,
  "expired_at": null,
  "expires_at": 1749221061,
  "failed_at": null,
  "finalizing_at": null,
  "in_progress_at": null,
  "metadata": null,
  "output_file_id": "",
  "request_counts": {
    "completed": 0,
    "failed": 0,
    "total": 0
  }
}
Track batch job:

2025-06-05 14:44:25.104873 Batch Id: batch_455d9b12-478c-4e8c-ae56-e3d1f48eca66,  Status: val

In [57]:
reload(promptizer2.library)
promptizer2.library.task_delimiter


'### PATIENTS SYMPTOMS ###'

In [ ]:
['gpt-4o', 'claude sonnet 3.7', 'nova micro']

{
  "id": "file-f30ad656cc5443bcb98d1185afc89035",
  "bytes": 22924,
  "created_at": 1749136752,
  "filename": "batch.jsonl",
  "object": "file",
  "purpose": "batch",
  "status": "processed",
  "expires_at": null,
  "status_details": null
}
Create batch job:

{
  "id": "batch_d6d271e9-7a90-435e-8ec7-f802977e83bb",
  "completion_window": "24h",
  "created_at": 1749136757,
  "endpoint": "/chat/completions",
  "input_file_id": "file-f30ad656cc5443bcb98d1185afc89035",
  "object": "batch",
  "status": "validating",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": null,
  "error_file_id": "",
  "errors": null,
  "expired_at": null,
  "expires_at": 1749223154,
  "failed_at": null,
  "finalizing_at": null,
  "in_progress_at": null,
  "metadata": null,
  "output_file_id": "",
  "request_counts": {
    "completed": 0,
    "failed": 0,
    "total": 0
  }
}
Track batch job:

2025-06-05 15:19:17.393769 Batch Id: batch_d6d271e9-7a90-435e-8ec7-f802977e83bb,  Status: validating
2025

In [ ]:
promptizer3.iterate()

In [107]:
def response_scores(responses, y):
    scores = []
    for r in responses:
        p = []
        for j in r['response']:
            jsponse = json.loads(re.search(r'\{.*\}', j, re.DOTALL).group(0))
        #if 'liklihood' in jsponse.keys():
        #    p.append(probability(jsponse['liklihood']))
        #else:
            p.append(probability(jsponse['likelihood generated by AI']))
        #    p.append(probability(jsponse['likelihood']))
        scores.append(roc_auc_score(y, p))
    return scores
train_scores = response_scores(train_responses, X_train['target'])
test_scores = response_scores(test_responses, X_test['generated'])
plt.plot(train_scores, label='train')
plt.plot(test_scores, label='test')
plt.legend()
ps = plt.show()


[0.8,
 0.7666666666666667,
 0.7333333333333333,
 0.7666666666666667,
 0.7333333333333333,
 0.7333333333333333,
 0.7333333333333333,
 0.8333333333333334,
 0.7333333333333333,
 0.7333333333333333]

In [100]:
accuracy = 0
for r, d in zip(responses.sort_values('custom_id')['response'], pd.DataFrame(df['train'])['output_text'][:30]):
    j = json.loads(re.sub(r'\}\}', '}', re.sub(r'\{\{', '{', r)))
    if j['disease'] == d:
        accuracy += 1
    else:
        print(j['disease'], ',', d)
accuracy/responses.shape[0]

pneumonia , common cold
psoriasis , drug reaction
impetigo , fungal infection
dengue , malaria
drug reaction , allergy
chicken pox , dengue
pneumonia , bronchial asthma
dengue , malaria


0.7333333333333333

### Bayesian Optimization of Random Prompts

In [101]:
import os
from matplotlib import pyplot as plt
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, Matern, DotProduct
from scipy.stats import ecdf, lognorm
from multiprocessing import Pool
from scipy.stats import norm

## Gaussian Process Regressor
- convert target to lognormal
- model process as log of converted target
- works better when target is actually normal
- use lognormal because expected value is "easy" to calculate

Univariate Expected Improvement

In [400]:
p = []
for j in responses['response']:
    jsponse = json.loads(re.search(r'\{.*\}', j, re.DOTALL).group(0))
    if 'liklihood' in jsponse.keys():
        p.append(probability(jsponse['liklihood']))
    else:
        p.append(probability(jsponse['likelihood'])) 

print(roc_auc_score(y_train, p))

# best_prompt = random_prompts_list.pop(best_idx)
# df = pd.concat([df, pd.DataFrame({'scores': roc_auc_score(y_train, p), 
#                                   'prompts': best_prompt}, index = [df.shape[0]])])
# best_embedding = random_prompt_embeddings.pop(best_idx)
# detection_prompt_embeddings.append(best_embedding)


0.4637276785714286


In [ ]:
# theres a few different ways to do sampling (iterative, flipping, different models, pure random)
# RAG: embed the RAG data into the 'text' in the training, test sets

In [ ]:
iterate_prompt(task_prompt, , examples, sample, paths)

{
  "id": "file-27704bf545054690841eeea1d74a0163",
  "bytes": 153758,
  "created_at": 1746979155,
  "filename": "74.jsonl",
  "object": "file",
  "purpose": "batch",
  "status": "processed",
  "expires_at": null,
  "status_details": null
}
Create batch job:

{
  "id": "batch_797a55be-37de-451e-8ca4-1926156c8249",
  "completion_window": "24h",
  "created_at": 1746979159,
  "endpoint": "/chat/completions",
  "input_file_id": "file-27704bf545054690841eeea1d74a0163",
  "object": "batch",
  "status": "validating",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": null,
  "error_file_id": "",
  "errors": null,
  "expired_at": null,
  "expires_at": 1747065555,
  "failed_at": null,
  "finalizing_at": null,
  "in_progress_at": null,
  "metadata": null,
  "output_file_id": "",
  "request_counts": {
    "completed": 0,
    "failed": 0,
    "total": 0
  }
}
Track batch job:

2025-05-11 15:59:19.654070 Batch Id: batch_797a55be-37de-451e-8ca4-1926156c8249,  Status: validating


In [32]:
import batch_bayesian_optimization as bbo
batch_size = 3
batch_idx, batch_mu, batch_sigma = bbo.create_batches(gpr, random_prompts_embeddings, 12, batch_size)
best_idx = bbo.get_best_batch(batch_mu, batch_sigma, batch_size)

In [36]:
batch_idx[best_idx]

[4, 24, 29]

In [37]:
from math import comb
comb(100, 4)

3921225